In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Credit Card Fraud Detection: Gözetimli Öğrenme Projesi\n",
    "\n",
    "## 1. Proje Tanımı\n",
    "\n",
    "- **Problem:** Kredi kartı işlemlerinin sahte (fraud) olup olmadığını tahmin etmek.\n",
    "- **Veri Seti:** [Kaggle - Credit Card Fraud Detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud)\n",
    "- **Amaç:** Gerçek hayatta finansal kayıpları önlemek için yeni işlemlerin sahte olup olmadığını tespit eden bir model geliştirmek.\n",
    "\n",
    "> **Not:** Bonus olarak, aynı veri seti üzerinde k-Means ile temel bir kümeleme de eklendi.\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Kütüphanelerin Yüklenmesi"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score\n",
    "from sklearn.preprocessing import StandardScaler\n",
    "from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve\n",
    "from sklearn.linear_model import LogisticRegression\n",
    "from sklearn.ensemble import RandomForestClassifier\n",
    "from sklearn.neighbors import KNeighborsClassifier\n",
    "from sklearn.tree import DecisionTreeClassifier\n",
    "from sklearn.svm import SVC\n",
    "from sklearn.cluster import KMeans\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n"
   ],
   "execution_count": 1,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Veri Setinin Yüklenmesi"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "# Kaggle'dan indirdiğiniz \"creditcard.csv\" dosyasını aynı dizine koyun\n",
    "df = pd.read_csv('creditcard.csv')\n",
    "df.shape"
   ],
   "execution_count": 2,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Keşifsel Veri Analizi (EDA)"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "df.head()"
   ],
   "execution_count": 3,
   "outputs": []
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "df.describe()"
   ],
   "execution_count": 4,
   "outputs": []
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "df.info()"
   ],
   "execution_count": 5,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Eksik veri var mı?**"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "df.isnull().sum().sum()"
   ],
   "execution_count": 6,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Sınıf Dağılımı**"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "sns.countplot(x='Class', data=df)\n",
    "plt.title('Sınıf Dağılımı (0=Normal, 1=Fraud)')\n",
    "plt.show()\n",
    "print(df['Class'].value_counts(normalize=True))"
   ],
   "execution_count": 7,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Veri ciddi şekilde dengesizdir (%0.17 fraud)."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Bazı özelliklerin dağılımı:**"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "plt.figure(figsize=(14,6))\n",
    "for i, col in enumerate(['V1', 'V2', 'V3', 'Amount']):\n",
    "    plt.subplot(1,4,i+1)\n",
    "    sns.histplot(df[col], bins=30, kde=True)\n",
    "    plt.title(col)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ],
   "execution_count": 8,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Korelasyon Matrisi:**"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "plt.figure(figsize=(12,9))\n",
    "sns.heatmap(df.corr(), cmap='coolwarm', vmin=-1, vmax=1, center=0)\n",
    "plt.title('Korelasyon Matrisi')\n",
    "plt.show()"
   ],
   "execution_count": 9,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Veri Ön İşleme"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "# Amount sütunu normalleştiriliyor\n",
    "scaler = StandardScaler()\n",
    "df['normAmount'] = scaler.fit_transform(df['Amount'].values.reshape(-1, 1))\n",
    "# Amount ve Time sütununu çıkarıyoruz (Time'ın anlamı yok, Amount normalize edildi)\n",
    "df = df.drop(['Time','Amount'], axis=1)"
   ],
   "execution_count": 10,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Eğitim ve test kümeleri:**"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "X = df.drop('Class', axis=1)\n",
    "y = df['Class']\n",
    "X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)"
   ],
   "execution_count": 11,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Model Seçimi & Hiperparametre Optimizasyonu\n",
    "- Lojistik Regresyon, Karar Ağacı, KNN ve Random Forest karşılaştırılacak"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "models = {\n",
    "    'Logistic Regression': LogisticRegression(),\n",
    "    'Decision Tree': DecisionTreeClassifier(),\n",
    "    'KNN': KNeighborsClassifier(),\n",
    "    'Random Forest': RandomForestClassifier(n_jobs=-1)\n",
    "}\n",
    "results = {}\n",
    "\n",
    "for name, model in models.items():\n",
    "    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')\n",
    "    results[name] = scores.mean()\n",
    "    print(f\"{name}: ROC-AUC = {scores.mean():.4f}\")"
   ],
   "execution_count": 12,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Random Forest en iyi ROC-AUC puanına sahip. Hiperparametre optimizasyonu yapılacak.**"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "param_grid = {\n",
    "    'n_estimators': [50, 100],\n",
    "    'max_depth': [5, 10, 20],\n",
    "    'min_samples_split': [2, 5],\n",
    "    'min_samples_leaf': [1, 2],\n",
    "}\n",
    "rf = RandomForestClassifier(n_jobs=-1, random_state=42)\n",
    "grid = GridSearchCV(rf, param_grid, cv=3, scoring='roc_auc', verbose=1, n_jobs=-1)\n",
    "grid.fit(X_train, y_train)\n",
    "print(\"En iyi parametreler:\", grid.best_params_)\n"
   ],
   "execution_count": 13,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Model Değerlendirme"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "best_rf = grid.best_estimator_\n",
    "y_pred = best_rf.predict(X_test)\n",
    "y_prob = best_rf.predict_proba(X_test)[:,1]\n",
    "\n",
    "print(\"Confusion Matrix:\")\n",
    "print(confusion_matrix(y_test, y_pred))\n",
    "print(classification_report(y_test, y_pred))\n",
    "print(f\"ROC-AUC: {roc_auc_score(y_test, y_prob):.4f}\")"
   ],
   "execution_count": 14,
   "outputs": []
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "fpr, tpr, _ = roc_curve(y_test, y_prob)\n",
    "plt.plot(fpr, tpr, label='Random Forest (AUC = %.4f)' % roc_auc_score(y_test, y_prob))\n",
    "plt.plot([0,1], [0,1], 'k--')\n",
    "plt.xlabel('False Positive Rate')\n",
    "plt.ylabel('True Positive Rate')\n",
    "plt.title('ROC Curve')\n",
    "plt.legend()\n",
    "plt.show()"
   ],
   "execution_count": 15,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Önemli Özellikler"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "feat_importances = pd.Series(best_rf.feature_importances_, index=X.columns)\n",
    "feat_importances.nlargest(10).plot(kind='barh')\n",
    "plt.title('En Önemli Özellikler')\n",
    "plt.show()"
   ],
   "execution_count": 16,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## BONUS: Gözetimsiz Öğrenme (k-Means ile Kümeleme)\n",
    "\n",
    "Yalnızca V1 ve V2 özellikleri kullanıldı (görselleştirme için)."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "kmeans = KMeans(n_clusters=2, random_state=42)\n",
    "subset = df[['V1','V2']].sample(5000, random_state=42)\n",
    "labels = kmeans.fit_predict(subset)\n",
    "plt.scatter(subset['V1'], subset['V2'], c=labels, cmap='coolwarm', alpha=0.5)\n",
    "plt.title('k-Means ile Kümeleme (V1-V2)')\n",
    "plt.xlabel('V1')\n",
    "plt.ylabel('V2')\n",
    "plt.show()"
   ],
   "execution_count": 17,
   "outputs": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# Credit Card Fraud Detection: Gözetimli Öğrenme Projesi\n',
    '\n',
    '## 1. Proje Tanımı\n',
    '\n',
    '- **Problem:** Kredi kartı işlemlerinin sahte (fraud) olup olmadığını tahmin etmek.\n',
    '- **Veri Seti:** [Kaggle - Credit Card Fraud Detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud)\n',
    '- **Amaç:** Gerçek hayatta finansal kayıpları önlemek için yeni işlemlerin sahte olup olmadığını tespit eden bir model geliştirmek.\n',
    '\n',
    '> **Not:** Bonus olarak, aynı veri seti üzerinde k-Means ile temel bir kümeleme de eklendi.\n']},
  {'cell_type': 'markdown',
   'metadata': {},
   'source': ['## 2. Kütüphanelerin Yüklenmesi']},
  {'cell_type': 'code',
   'metadata': {},
   'source': ['import pandas as pd\n',
    'import numpy as np\n',
    'import matplotlib.pyplot as plt\n',
    'import seaborn as sns\n',
    'from sklearn.model_selection import train_test_split, GridSearchCV